In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pypfopt.efficient_frontier import EfficientFrontier
import ipywidgets as widgets

# Set random seed for reproducibility
np.random.seed(1)

# Create a DataFrame with 5 data series, each containing 200 observations
data = {
    'Asset 1': np.random.normal(1, 3.5, 150),
    'Asset 2': np.random.normal(1.5, 4.5, 150),
    'Asset 3': np.random.normal(2, 5, 150),
    'Asset 4': np.random.normal(2.5, 5.5, 150),
    'Asset 5': np.random.normal(3, 6, 150)
}
df = pd.DataFrame(data)
# Put data into cumulative terms
df = np.cumprod(1+df/100)

end_date = pd.Timestamp.today().to_period('Q').end_time
start_date = end_date - pd.offsets.QuarterEnd(len(df)-1)
date_range = pd.date_range(start=start_date, end=end_date, freq='Q')

# Set the index of the DataFrame
df.index = date_range.strftime('%b-%Y')

df_ret = df.pct_change().dropna()

def ef_plotter(ef_sig, ef_ret, sharpe_sig, sharpe_ret):

    fig,ax = plt.subplots(figsize=(5.5,4))

    ax.plot(ef_sig, ef_ret, lw=0.75, label='Efficient frontier', color='#004552')

    ax.scatter(sharpe_sig, sharpe_ret, facecolor='white', edgecolor='#004552', lw=1, s=40, zorder=10, label='Tangency portfolio')

    ax.legend()

    ax.set_xlabel('Annualised return standard deviation (%)')
    ax.set_ylabel('Annualised return (%)', rotation=0, ha='left', va='bottom')
    ax.yaxis.set_label_coords(0,1)

    ax.set_title('Risk vs return', pad=20)


In [71]:
import numpy as np
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from pypfopt import EfficientFrontier

# Assuming df_ret is already defined
formatted_dates = df_ret.index

widget_width = '400px'

start_drop_widget = widgets.Dropdown(
    options=formatted_dates[:-15],
    value=formatted_dates[0],
    description='Window start:',
    disabled=False,
    layout=widgets.Layout(width=widget_width),
    style={'description_width': 'initial'}
)

end_drop_widget = widgets.Dropdown(
    options=formatted_dates[15:],
    value=formatted_dates[-1],
    description='Window end:',
    disabled=False,
    layout=widgets.Layout(width=widget_width),
    style={'description_width': 'initial'}
)

# Create a list of checkboxes for each asset
asset_checkboxes = []
for asset in df_ret.columns:
    asset_checkboxes.append(
        widgets.Checkbox(
            value=True,
            description=asset,
            disabled=False,
            layout=widgets.Layout(width='auto', margin='0 5px 0 0'),
            style={'description_width': 'initial'}
        )
    )

# Callback function to ensure at least 15 observations are included in the calculation
def update_end_options(*args):
    start_index = formatted_dates.tolist().index(start_drop_widget.value)
    end_options = formatted_dates[start_index + 15:]
    if len(end_options) < 15:
        end_drop_widget.options = end_options
        if end_drop_widget.value not in end_options:
            end_drop_widget.value = end_options[0]

# Attach the callback function to the start dropdown widget
start_drop_widget.observe(update_end_options, names='value')
# Initial call to set the correct end options
update_end_options()

def plotter(start_drop, end_drop, **assets):
    selected_assets = [asset for asset, selected in assets.items() if selected]
    if len(selected_assets) < 2:
        raise ValueError("Please select at least two assets.")
    
    start_date = start_drop 
    end_date = end_drop 

    t_data = df_ret.loc[start_date:end_date, selected_assets]
    t_rf = 3.5
    t_mu = 100 * (((1 + t_data.mean()) ** 4 - 1))
    t_cov = t_data.cov() * 4
    t_assets = t_data.columns

    t_min = t_mu.min()
    t_max = t_mu.max()
    ret_steps = np.linspace(t_min, t_max, 100)

    ef_w = []
    ef_sig = []
    ef = EfficientFrontier(t_mu, t_cov)
    for t_ret_step in ret_steps:
        try:
            ef.efficient_return(t_ret_step)
            t_w = ef.clean_weights()
            t_w = [t_w[i] for i in t_assets]
            ef_sig.append(t_w @ t_cov @ t_w)
            ef_w.append(t_w)
        except:
            np.nan

    ef_w = pd.DataFrame(ef_w, columns=t_assets)
    ef_ret = ef_w @ t_mu
    ef_sig = np.multiply(np.sqrt(ef_sig), 100)

    # Get Sharpe
    ef = EfficientFrontier(t_mu, t_cov)
    ef.max_sharpe(risk_free_rate=t_rf)
    t_w = ef.clean_weights()
    sharpe_w = t_w = [t_w[i] for i in t_assets]
    sharpe_ret = sharpe_w @ t_mu
    sharpe_sig = np.multiply(np.sqrt(sharpe_w @ t_cov @ sharpe_w), 100)
    sharpe = (sharpe_ret - t_rf) / sharpe_sig

    # Plotting
    ef_fig = ef_plotter(ef_sig, ef_ret, sharpe_sig, sharpe_ret)

# Create the interactive plot without displaying the widgets
widget_dict = {
    'start_drop': start_drop_widget,
    'end_drop': end_drop_widget
}

# Add each asset checkbox to the widget dictionary
for asset_checkbox in asset_checkboxes:
    widget_dict[asset_checkbox.description] = asset_checkbox

interactive_plot = widgets.interactive_output(plotter, widget_dict)

# Manually display the widgets
display(widgets.VBox(children=[start_drop_widget, end_drop_widget, widgets.HBox(children=asset_checkboxes)]))

# Display the interactive plot
display(interactive_plot)

Output()